In [4]:
from google.colab import auth
from datetime import datetime
import os
import pandas as pd
!git clone https://github.com/AIStream-Peelout/flow-forecast.git -b probablistic_gr
os.chdir("flow-forecast")
!pip install -r requirements.txt
!python setup.py develop

Cloning into 'flow-forecast'...
remote: Enumerating objects: 18191, done.
remote: Counting objects: 100% (604/604), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 18191 (delta 367), reused 511 (delta 314), pack-reused 17587
Receiving objects: 100% (18191/18191), 4.72 MiB | 16.22 MiB/s, done.
Resolving deltas: 100% (13076/13076), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 5.0 MB/s 
     |████████████████████████████████| 6.0 MB 21.6 MB/s 
     |████████████████████████████████| 1.8 MB 57.1 MB/s 
     |████████████████████████████████| 28.8 MB 75.7 MB/s 
     |████████████████████████████████| 1.2 MB 43.0 MB/s 
     |████████████████████████████████| 201 kB 52.3 MB/s 
     |████████████████████████████████| 2.8 MB 46.2 MB/s 
     |████████████████████████████████| 182 kB 68.4 MB/s 
     |████████████████████████████████| 166 kB 54.7 MB/s 
     |████████████

/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:530: UserWarning: Normalizing '0.999998dev' to '0.999998.dev0'
  warnings.warn(tmpl.format(**locals()))
running develop
/usr/local/lib/python3.7/dist-packages/setuptools/command/easy_install.py:147: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  EasyInstallDeprecationWarning,
/usr/local/lib/python3.7/dist-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,
running egg_info
creating flood_forecast.egg-info
writing flood_forecast.egg-info/PKG-INFO
writing dependency_links to flood_forecast.egg-info/dependency_links.txt
writing requirements to flood_forecast.egg-info/requires.txt
writing top-level names to flood_forecast.egg-info/top_level.txt
writing manifest file 'flood_forecast.egg-info/SOURCES.txt'
adding licens

In [5]:
from flood_forecast.basic.linear_regression import simple_decode
from flood_forecast.basic.gru_vanilla import VanillaGRU

In [84]:
from ast import Raise
import torch
from typing import Type
def simple_decode(model: Type[torch.nn.Module],
                  src: torch.Tensor,
                  max_seq_len: int,
                  real_target: torch.Tensor,
                  start_symbol=None,
                  output_len=1,
                  device='cpu',
                  unsqueeze_dim=1,
                  meta_data=None,
                  multi_targets=1,
                  use_real_target: bool = True,
                  probabilistic: bool = False,
                  scaler=None) -> torch.Tensor:
    """
    :model a PyTorch model to be used for decoding
    :src the source tensor
    :the max length sequence to return
    :real_target the actual target values we want to forecast (don't worry they are masked)
    :start_symbol used to match the function signature of greedy_decode not ever used here though.
    :output_len potentially used to forecast multiple steps at once. Not implemented yet though.
    :device used to to match function signature
    :multi_task int: Multitask return will always be (batch_size, output_len, multi_targets)
    :returns a torch.Tensor of dimension (B, max_seq_len, M)
    Currently does not work with batch_size of greater than 1
    """
    real_target = real_target.float()
    real_target2 = real_target.clone()
    # Use last value
    ys = src[:, -1, :].unsqueeze(unsqueeze_dim)
    ys_std_dev = []
    upper_out = []
    lower_out = []
    handle_gauss = False
    for i in range(0, max_seq_len, output_len):
        residual = output_len if max_seq_len - output_len - i >= 0 else max_seq_len % output_len
        with torch.no_grad():
            if meta_data:
                out = model(src, meta_data).unsqueeze(2)
            else:
                out = model(src)
                if isinstance(out, tuple):
                    # out, up, lower = handle_gaussian_loss(out)
                    print(up)
                    upper_out.append(up[:, :residual])
                    lower_out.append(lower[:, :residual])
                    handle_gauss = True
                elif probabilistic:
                    out_std = out.stddev.detach()
                    out = out.mean.detach()
                    print(ys_std_dev)
                    ys_std_dev.append(out_std[:, 0:residual])
                elif multi_targets < 2:
                    out = out.unsqueeze(2)
            if scaler:
              print("de")
                # handle_no_scaling(scaler, out, multi_targets)
            if not isinstance(out, torch.Tensor):
                out = torch.from_numpy(out)
            if output_len == 1:
                rea_target2 = real_target2[:, i, 0:multi_targets]
                print(rea_target2.shape)
                real_target2[:, i, 0:multi_targets] = out[:, 0]
                src = torch.cat((src[:, 1:, :], real_target2[:, i, :].unsqueeze(1)), 1)
                ys = torch.cat((ys, real_target2[:, i, :].unsqueeze(1)), 1)
            else:
                # residual = output_len if max_seq_len - output_len - i >= 0 else max_seq_len % output_len
                if output_len != out.shape[1]:
                  raise ValueError("Output length does not equal the output shape")
                real_target2[:, i:i + residual, 0:multi_targets] = out[:, :residual]
                src = torch.cat((src[:, residual:, :], real_target2[:, i:i + residual, :]), 1)
                ys = torch.cat((ys, real_target2[:, i:i + residual, :]), 1)
    if probabilistic:
        ys_std_dev = torch.cat(ys_std_dev, dim=1)
        return ys[:, 1:, 0:multi_targets], ys_std_dev
    if handle_gauss:
        return torch.cat(upper_out, dim=1), torch.cat(lower_out, dim=1), ys[:, 1:, 0:multi_targets]
    else:
        return ys[:, 1:, 0:multi_targets]


In [87]:
# A Gru
m = VanillaGRU(3, 30, 1, 2, 0.2, forecast_length=2, probabilistic=True)
simple_decode(m1, torch.rand(1, 20, 3), 5, torch.rand(1, 20, 3), output_len=2, probabilistic=True, multi_targets=1)[0].shape

torch.Size([1, 2, 1])
[]
torch.Size([1, 2, 1])
[tensor([[[0.0100],
         [0.0100]]])]
torch.Size([1, 2, 1])
[tensor([[[0.0100],
         [0.0100]]]), tensor([[[0.0100],
         [0.0100]]])]


torch.Size([1, 5, 1])

In [35]:
m(torch.rand(4, 20, 3))

Normal(loc: torch.Size([4, 1]), scale: torch.Size([4, 1]))

In [98]:
class VanillaGRU1(torch.nn.Module):
    def __init__(self, n_time_series: int, hidden_dim: int, num_layers: int, n_target: int, dropout: float,
                 forecast_length=1, use_hidden=False, probabilistic=False):
        """
        Simple GRU to preform deep time series forecasting.

        :param n_time_series: The number of time series present in the data
        :type n_time_series:
        :param hidden_dim:
        :type hidden_dim:

        Note for probablistic n_targets must be set to two and actual multiple targs are not supported now.
        """
        super(VanillaGRU1, self).__init__()

        # Defining the number of layers and the nodes in each layer
        self.layer_dim = num_layers
        self.hidden_dim = hidden_dim
        self.hidden = None
        self.use_hidden = use_hidden
        self.forecast_length = forecast_length
        self.probablistic = probabilistic

        # GRU layers
        self.gru = torch.nn.GRU(
            n_time_series, hidden_dim, num_layers, batch_first=True, dropout=dropout
        )

        # Fully connected layer
        self.fc = torch.nn.Linear(hidden_dim, n_target)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward function for GRU

        :param x: torch of shape
        :type model: torch.Tensor
        :return: Returns a tensor of shape (batch_size, forecast_length, n_target) or (batch_size, n_target)
        :rtype: torch.Tensor
        """
        # Initializing hidden state for first input with zeros
        if self.hidden is not None and self.use_hidden:
            h0 = self.hidden
        else:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, self.hidden = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -self.forecast_length:, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        if self.probablistic:
            mean = out[..., 0][..., None]
            print(mean.shape)
            std = torch.clamp(out[..., 1][..., None], min=0.01)
            y_pred = torch.distributions.Normal(mean, std)
            return y_pred
        return out


In [96]:
m1 = VanillaGRU1(3, 30, 1, 1, 0.2, forecast_length=2)

In [97]:
m1(torch.rand(4, 20, 3)).shape

torch.Size([4, 2, 1])

In [99]:
m1.fc.out_features

1